<a href="https://colab.research.google.com/github/SDS-AAU/SDS-master/blob/master/M2/notebooks/M2_Case_ETF_Holdings_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Making a map of ETF Holdings with Bipartite Networks

Read the original project description by Andrey Babynin [here](https://towardsdatascience.com/etf2vec-my-story-about-trying-to-extact-narrative-from-etf-holdings-5a5f355ce3c4)






In [1]:
!mkdir ETF

In [2]:
!wget https://github.com/andreybabynin/ETF/raw/master/2019-7-7.zip

--2020-11-02 17:48:42--  https://github.com/andreybabynin/ETF/raw/master/2019-7-7.zip
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/andreybabynin/ETF/master/2019-7-7.zip [following]
--2020-11-02 17:48:42--  https://raw.githubusercontent.com/andreybabynin/ETF/master/2019-7-7.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24282593 (23M) [application/zip]
Saving to: ‘2019-7-7.zip’

2019-7-7.zip        100%[===================>]  23.16M  40.6MB/s    in 0.6s    

2020-11-02 17:48:43 (40.6 MB/s) - ‘2019-7-7.zip’ saved [24282593/24282593]



In [ ]:
!unzip -d /content/ETF 2019-7-7.zip

In [4]:
import glob
import io
import re
import pandas as pd
import numpy as np
import networkx as nx
from networkx.algorithms import bipartite

In [5]:
paths = glob.glob('/content/ETF/*.csv')

In [6]:
paths[:10]

['/content/ETF/IEMD_holdings.csv',
 '/content/ETF/DPYG_holdings.csv',
 '/content/ETF/CEUU_holdings.csv',
 '/content/ETF/CEMG_holdings.csv',
 '/content/ETF/IWSZ_holdings.csv',
 '/content/ETF/JKI_holdings.csv',
 '/content/ETF/IUGA_holdings.csv',
 '/content/ETF/SMUD_holdings.csv',
 '/content/ETF/EMUL_holdings.csv',
 '/content/ETF/IWX_holdings.csv']

In [7]:
edgelist = []
isin_list = []

for path in paths:
  fund = re.findall(r'ETF/(.*?)\_', path)
  string = open(path).read()
  start = string.find('Issue')
  if start == -1:
    start = string.find('Name')
  holdings = pd.read_csv(io.StringIO(string[start:]))
  if 'ISIN' not in holdings.columns or 'Weight (%)' not in holdings.columns:
    continue
  holdings.dropna(subset=['ISIN'], inplace=True)
  holdings = holdings[holdings.ISIN != '-']
  
  isin_list.extend(holdings[['ISIN','Name']].values)
  holdings = holdings[['ISIN', 'Weight (%)']]
  edges = [(fund[0], isin, weight) for isin, weight in holdings.values]

  edgelist.extend(edges)

In [8]:
len(edgelist)

437950

In [9]:
isin_df = pd.DataFrame(isin_list)

In [10]:
isin_df.head()

,0,1
0,CH0012005267,NOVARTIS AG
1,CH0012032048,ROCHE HOLDING PAR AG
2,CH0038863350,NESTLE SA
3,DE0007164600,SAP
4,FR0000121014,LVMH


In [11]:
isin_df.drop_duplicates(subset=[0], inplace=True)
isin_df.columns = ['ISIN','Name']

In [28]:
B = nx.Graph()

In [29]:
c0 = set([c[0] for c in edgelist])
c1 = set([c[1] for c in edgelist])

In [30]:
B.add_nodes_from(c0, bipartite=0)
B.add_nodes_from(c1, bipartite=1)
B.add_weighted_edges_from(edgelist)

In [31]:
degrees = pd.Series(dict(B.degree()))

In [32]:
perc = np.percentile(degrees, 95)

In [33]:
degrees[degrees < perc]

EXV2            19
IBTM            20
EXHB             7
EXH6            22
ITKY            18
                ..
US25271CAN20     8
US918204AT53     5
US679111UL65     1
US9282541013    14
US593338AG17     1
Length: 43861, dtype: int64

In [34]:
degrees[degrees < perc].index & c1

Index(['JP3893700009', 'US052403FA56', 'FR0013409844', 'TW0003682001',
       'US566795KN91', 'US29874QDA94', 'US88579YAR27', 'US1170431092',
       'US677522GJ72', 'GB0006650450',
       ...
       'GB00B8460Z43', 'CA15135U1093', 'IL0006110121', 'US744448CL30',
       'US9174881089', 'US25271CAN20', 'US918204AT53', 'US679111UL65',
       'US9282541013', 'US593338AG17'],
      dtype='object', length=43763)

In [36]:
B.remove_nodes_from(degrees[degrees < perc].index & c1)

In [37]:
B_eq = bipartite.weighted_projected_graph(B, degrees[degrees >= perc].index & c1)

In [38]:
e99 = np.percentile([d['weight'] for u,v,d in B_eq.edges(data=True)], 99)

In [39]:
B_eq_99 = nx.edge_subgraph(B_eq, [(u,v) for u,v,d in B_eq.edges(data=True) if d['weight'] > e99])

In [40]:
print(nx.info(B_eq_99))

Name: 
Type: Graph
Number of nodes: 665
Number of edges: 10196
Average degree:  30.6647


In [41]:
isin_df

,ISIN,Name
0,CH0012005267,NOVARTIS AG
1,CH0012032048,ROCHE HOLDING PAR AG
2,CH0038863350,NESTLE SA
3,DE0007164600,SAP
4,FR0000121014,LVMH
...,...,...
437338,DE000HV2ANF3,UNICREDIT BANK AG MTN RegS
437340,DE0001141711,GERMANY (FEDERAL REPUBLIC OF) RegS
437341,DE0001104719,GERMANY (FEDERAL REPUBLIC OF) RegS
437342,DE0001104701,GERMANY (FEDERAL REPUBLIC OF) RegS


In [42]:
attributes_dict=dict(isin_df.iloc[:,0:].values)

In [43]:
nx.set_node_attributes(B_eq_99, attributes_dict, 'Name')

In [44]:
nx.write_gexf(B_eq_99,'isin.gexf')